# Projet Optimisation (groupe 6) - Benjamin Piet & Damien Capéraa

## 2. Etude et resolution numerique

### Question 1

On se retrouve donc avec un problème de la forme
$min(\frac{1}{2}p^TAp-b^Tp|C^Tp-d<=0)$\
On note $f(p)=\frac{1}{2}p^TAp-b^Tp$ et $c(p)=C^Tp-d$. \
\
Tout d'abord, la fonction f est ***fortement convexe*** car la matrice A est symmetrique definie positive. En effet, si on calcule ses valeurs propres via python:

In [3]:
import numpy as np
A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
     [0, 0, -0.0031, 0.0054]])
valeurs_propres = np.linalg.eig(A)[0]
print(valeurs_propres)

[0.02753417 0.00496583 3.0825     0.0405    ]


Toutes les valeurs propres de A étant strictement positives, A est symétrique définie positive. Par conséquent, la fonction de coût quadratique f est convexe.\
De plus, l'ensemble de recherche est convexe, car les contraintes sont toutes affines.
On a donc ***l'existence d'un unique minimum global***.\
\
Le conditionnement du problème vaut :
$$K(A) = \frac{max(Sp(A))}{min(Sp(A))} = 620,7$$
**faire des recherches sur le conditionnement**
\
Au vu des divers hypothèses et de la forme du problème, on propose d'utiliser ***l'algorithme des contraintes actives*** que nous avons vu dans le poly d'optimisation p.39.

**Etape 2 (a) de l'algorithme : résolution du problème d'otpimisation sous contraintes d'égalité** \
L'étape consiste à résoudre $\displaystyle \min_{c_i^Tp = 0, i \in W^k} \frac{1}{2}p^TGp + (Gx^k + d)^Tp$.
On introduit le Lagrangien du problème $\mathcal{L}(p, \lambda) = \frac{1}{2}p^TGp + (Gx^k + d)^Tp + \lambda^TD^Tp$ ou $D$ est la matrice composée des colonnes $c_i$ de C pour $i \in W^k$.
Un point stationnaire du Lagrangien sans contraintes est un point stationnaire de la fonction coût sous contraintes : on cherche donc le point $(p^*, \lambda^*)$ qui annule les deux dérivées partielles de $\mathcal{L}$.
$$
\begin{align}
\frac{\partial\mathcal{L}}{\partial\lambda}(p^*, \lambda^*) &= 0 = D^Tp \\
\frac{\partial\mathcal{L}}{\partial p}(p^*, \lambda^*) &= 0 = Ap + (Ax^k + b) + D\lambda \\
\end{align}$$
On réécrit le problème sous forme matricielle car c'est la formulation qui peut être exploitée avec la fonction numpy `np.linalg.solve`.
$$
\begin{pmatrix} D^T&0 \\
A&D\\
\end{pmatrix}
\begin{pmatrix} p \\
\lambda \\
\end{pmatrix}
=
\begin{pmatrix} 0 \\
-Ax^k - b \\
\end{pmatrix}
$$

In [1]:
#Algorithme des contraintes actives
import numpy as np

A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
     [0, 0, -0.0031, 0.0054]])
b = np.array([[2671], [135], [103], [19]])
C = np.array([[-0.0401, -0.0162, -0.0039, 0.0002], \
    [-0.1326, -0.0004, -0.0034, 0.0006], [1.5413, 0, 0, 0], \
    [0, 0.0203, 0, 0], [0, 0, 0.0136, -0.0015], \
    [0, 0, -0.0016, 0.0027], [0.0160, 0.0004, 0.0005, 0.0002]]).transpose()
d = np.array([[-92.6], [-29.0], [2671], [135], [103], [19], [10]])
eps = 1e-8 # paramètre donnant la valeur maximale des composantes
# de la direction pk pour que pk ne soit pas considérée comme nulle.

def f(p):
    return float(0.5 * np.matmul(p.transpose(), np.matmul(A, p)) - np.dot(b.transpose(), p))

def c(p):
    return np.matmul(C.transpose(), p) - d

def xk_is_solution(xk, Wk):
    ''' Etape 1 de l'algorithme des contraintes actives.
    Détermine, grâce aux conditions KKT, si xk est une solution du problème
    d'optimisation restreint aux contraintes de Wk.'''
    grad_contraintes = C[:, Wk]
    rg = np.linalg.matrix_rank(grad_contraintes)
    if rg == len(Wk):
        # Les contraintes actives sont qualifiées
        Lambda = np.linalg.solve(-grad_contraintes, xk)
        return all([Lambda[i, 0] > 0 for i in range(Lambda.shape[0])]), Lambda
    else:
        # Les contraintes actives ne sont pas qualifiées
        return False, None

W0 = [0, 1, 2, 3] # arbitraire mais permet de n'avoir qu'un point de départ possible
# car le système à résoudre contient 4 équations pour 4 inconnues.
# Recherche de p0 où ces 4 contraintes sont actives
CC = C[:, W0]
dd = d[W0, :]
x0 = np.linalg.solve(CC.transpose(), dd)

def contraintes():
    xk = x0
    Wk = W0
    while True:
        if xk_is_solution(xk, Wk)[0]:
            return xk, Wk
        # (a)
        '''
        L'étape (a) consiste à résoudre un problème d'optimisation sous
        contraintes d'égalité. On cherche alors (p*, Lambda*) point stationnaire
        du lagrangien associé. Cette recherche, comme présenté dans le notebook,
        aboutit à la résolution d'un système linéaire.
        '''
        D = C[:, Wk]
        E_ligne_0 = np.concatenate((D.transpose(), np.zeros((D.shape[1], D.shape[1]))), axis=1)
        E_ligne_1 = np.concatenate((A, D), axis=1)
        E = np.concatenate((E_ligne_0, E_ligne_1), axis=0)
        F = np.concatenate((np.zeros((D.shape[1], 1)), -np.matmul(A, xk) + b), axis=0)
        X = np.linalg.solve(E, F)
        pk = X[0:4, :]
        if any([abs(pk[i, 0]) > eps for i in range(4)]):
            # (b) : pk != 0
            W_barre = [i for i in range(7) if i not in Wk]
            L, indices = [], []
            for i in W_barre:
                c_i = C[:,i]
                if np.dot(c_i, pk) > 0:
                    L.append((d[i, 0] - np.dot(c_i, xk))/np.dot(c_i, pk))
                    indices.append(i)
            if L != []:
                alphak = min(1, min(L))
            else:
                alphak = 1
            xk = xk + alphak*pk
            if alphak < 1:
                j = indices[L.index(min(L))]
                Wk.append(j)
        else:
            # (c) ! pk = 0
            booleen, Lambda = xk_is_solution(xk, Wk)
            if booleen:
                return xk, Wk
            Lambda = Lambda.tolist()
            c = Lambda.index(min(Lambda))
            del Wk[c]
    return xk, Wk

In [2]:
contraintes()

LinAlgError: Last 2 dimensions of the array must be square